## CD to curent directory

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [2]:
import sys
ROOT_PATH = '/content/drive/My Drive/university/projects/NLP/project/code/'
#ROOT_PATH = '/content/drive/My Drive/code/'
sys.path.append(ROOT_PATH)

%cd /content/drive/My Drive/university/projects/NLP/project/code/
#%cd /content/drive/My Drive/code/

/content/drive/My Drive/university/projects/NLP/project/code


## Imports

In [3]:
import torch
from torch import nn,optim
import torch.nn.functional as F
import os,random,csv
import numpy as np
!pip install empath
from utils.EarlyStopping import EarlyStopping
from pathlib import Path
from IPython.display import display
from torchtext import data
from torchtext.vocab import Vectors, GloVe
csv.field_size_limit(sys.maxsize)
from os import mkdir
from os.path import isdir
import pandas as pd
import itertools
from sklearn.metrics import precision_recall_fscore_support
import datetime

     |████████████████████████████████| 61kB 2.0MB/s 
  Created wheel for empath: filename=empath-0.89-cp36-none-any.whl size=57824 sha256=db29148b6b6dad05fd461a3818050e9effe4279a43a9852651e4911c372971a8
  Stored in directory: /root/.cache/pip/wheels/84/ea/2f/2bc54d4f9985ce61753ebc5b00cb2df51d855589267c667308
Successfully built empath


## Models

### BiLSTM

In [0]:
class BiLstmlClassification(torch.nn.Module):
  """
  Implements a BiLSTM network with an embedding layer and single hidden layer.
  """
  def __init__(self, config, pretrained_embeddings, feat_names=[],pad_index=1):
    super(BiLstmlClassification, self).__init__()
    """
    Arguments
    ---------
    batch_size : Size of the batch which is same as the batch_size of the data returned by the TorchText BucketIterator
    hidden_size : Size of the hidden_state of the LSTM
    # vocab_size : Size of the vocabulary containing unique words
    # embed_size : Embeddding dimension of GloVe word embeddings
    # pretrained_embeddings : Pre-trained GloVe word_embeddings which 
    we will use to create our word_embedding look-up table 
    """

    # parameters
    self.n_classes = config.n_classes
    self.avg = config.avg
    self.embedding_dim = config.embedding_dim
    self.hidden_size = config.hidden_size
    self.nlayers = config.hidden_layer
    self.vocab_size, self.emb_dim = pretrained_embeddings.shape
    self.device = config.device
    self.doc_embedding = config.doc_embedding
    self.embedding_type = config.embedding_type
    
    # layers:
    self._dropout = torch.nn.Dropout(config.dropout)
    self.word_embeddings = torch.nn.Embedding.from_pretrained(pretrained_embeddings.to(config.device),
                                                              freeze=True, 
                                                              padding_idx=pad_index)
    self.bn_embedding_dim = nn.BatchNorm1d(self.embedding_dim)
    self.lstm = torch.nn.LSTM(self.embedding_dim,
                              self.hidden_size // 2,
                              num_layers=self.nlayers,
                              bidirectional=True,
                              batch_first=True)  # when input data is of shape (batch_size, seq_len, features)
    self.meta_feat_names = []
    self.W_s1 = nn.Linear(self.hidden_size, 50)
    self.W_s2 = nn.Linear(50, 30)
    self.fc_layer = nn.Linear(30*self.hidden_size, 2000)
    if "self_attention" in config.model_name:
      self.hidden2tag = nn.Linear(2000, self.n_classes)
    else:
      self.hidden2tag = torch.nn.Linear((config.hidden_size//2)*config.hidden_layer*2, self.n_classes) # Maps the output of the LSTM into tag space.
  
  def pooling(self, output, operation_dim=1, **kwargs):
    # The shape is now (batch_size, emb_dim)
    if self.embedding_type=='avg':
      doc_embedding = torch.mean(output, dim=operation_dim,keepdim=False)
    elif self.embedding_type=='max':
      doc_embedding = torch.max(output, dim=operation_dim,keepdim=False)[0]
    return doc_embedding

  def n_gram_pooling(self, output, seq_length, n_gram_length=5, operation_dim=1, **kwargs):
      """
      output.size: (batch_size, seq_len, embedding_dim)
      n_gram の max-pooling vector
      Returns:
          np.ndarray: shape = (n_embedded_dim, )
      """
      gram_vectors = []
      for i in range(max(1, seq_length - n_gram_length)):
        gram_i = output[:,i:i + n_gram_length,:]
        gram_mean = torch.mean(gram_i, dim=operation_dim) # tranform from dim: (batch_size, seq_len, embedding_dim) to dim: (batch_size, hidden_size)
        gram_vectors.append(gram_mean)
      gram_vectors = torch.stack(gram_vectors) # gram_vectors dim:(n_gram, batch_size, hidden_size)
      
      #max/avg pooling: dim:(batch_size, hidden_size)
      if self.embedding_type=='avg':
        pooling = torch.mean(gram_vectors, dim=0)
      elif self.embedding_type=='max':
        pooling = torch.max(gram_vectors, dim=0)[0]
      return pooling
		
  def attention_net(self, output, final_feature_map, **kwargs):
    """ 
    Use attention to compute soft alignment score corresponding between each of the hidden_state and the last hidden_state of the LSTM.
    We will be using torch.bmm for the batch matrix multiplication.
    Arguments
    ---------
    output : Final output of the LSTM which contains hidden layer outputs for each sequence.
    final_feature_map : Final and First time-step hidden state (h_n) of the LSTM
    ---------
    Returns :performs attention mechanism by first computing weights for each of the sequence present in output and then computing the new hidden state.
    """
    #feature_map.size = (batch_size, hidden_size)
    attn_weights = torch.bmm(output, final_feature_map.unsqueeze(2)).squeeze(2) #(batch_size, num_seq)
    soft_attn_weights = F.softmax(attn_weights, 1) #(batch_size, num_seq)
    new_hidden_state = torch.bmm(output.transpose(1, 2), soft_attn_weights.unsqueeze(2)).squeeze(2) # (batch_size, hidden_size)
    return new_hidden_state
	
  def self_attention_net(self, output, **kwargs):
    """
		self attention mechanism to produce a matrix embedding of the input sentence in which every row represents an
		encoding of the input sentence but giving an attention to a specific part of the sentence. 
    We will use 30 embedding of the input sentence and then finally we will concatenate all the 30 sentence embedding vectors and connect it to a fully 
		connected layer of size 2000 which will be connected to the output layer of size 2 returning logits for the final number of classes
		Arguments
		---------
		output = A tensor containing hidden states corresponding to each time step of the LSTM network.
		---------
		Returns : Final Attention weight matrix for all the 30 different sentence embedding in which each of 30 embeddings give
				  attention to different parts of the input sentence.
		"""
    # output.size() = (batch_size, num_seq, hidden_size)
    attn_weight_matrix = self.W_s2(torch.tanh(self.W_s1(output)))
    attn_weight_matrix = attn_weight_matrix.permute(0, 2, 1) #(batch_size, 30, num_seq)
    attn_weight_matrix = F.softmax(attn_weight_matrix, dim=2)
    hidden_matrix = torch.bmm(attn_weight_matrix, output) #(batch_size, 30, hidden_size)
    # concatenate the hidden_matrix and apply fully connected layer. #(batch_size, 2000)
    fc_out = self.fc_layer(hidden_matrix.view(-1, hidden_matrix.size()[1]*hidden_matrix.size()[2]))
    return fc_out

  def forward(self, text, text_lengths, **kwargs):
    batch_size, seq_length = text.shape[0], text.shape[1] # dim: batch_size x seq_len
    #self.h, self.c_n = self.init_hidden(batch_size)

    # 1. embed the input - 
    embeds = self.word_embeddings(text.long()) # dim: (batch_size x seq_len x embedding_dim)
    drop = self._dropout(embeds)

    # packed sequence - remove pads
    packed_embedded = nn.utils.rnn.pack_padded_sequence(drop, text_lengths, batch_first=True, enforce_sorted=False)
        
    # 2. Run through RNN - Dim transformation: (batch_size, seq_len, embedding_dim) -> (batch_size, seq_len, hidden_dim)
    lstm_out_packed, (h_n, c_n) = self.lstm(packed_embedded) # If `(h_0, c_0)` is not provided, both **h_0** and **c_0** default to zero
  
    # unpack, recover padded sequence
    output, input_sizes = nn.utils.rnn.pad_packed_sequence(lstm_out_packed, batch_first=True, total_length=seq_length)

    # 3. doc embedding of the output of BiLSTM - Dim transformation: (batch_size, seq_len, hidden_dim) ->  (batch_size, hidden_dim)
    final_feature_map =self._dropout(torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim = 1)) #concat the final forward and backward hidden state
    output = self.doc_embedding(self, output = output, final_feature_map = final_feature_map, seq_length=seq_length)
    
    # 4. Project to tag space
    tag_space = self.hidden2tag(output)  # Dim transformation: (batch_size, hidden_dim)->(batch_size x num_tags)

    return tag_space

# Trainer

In [0]:
#%debug
class Trainer:
  def __init__(self, model, config):
    self.device = config.device
    self.exp_num = config.exp_num
    self.model = model
    self.model.to(self.device)
    self.model_name = config.model_name
    self.seed = config.seed
    self.lr = config.lr
    self.epochs = config.epochs
    self.save_model = config.save_model
    self.upload_model = config.upload_model
    self.model_weights_path = config.model_weights_path
    self.batch_size = config.batch_size
    self.momentum = config.momentum
    self.save_points = config.save_points
    self.best_loss = float("inf")
    self.best_acc = float("-inf")
    self.soft = torch.nn.Softmax(dim=1)
    self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr, weight_decay=config.weight_decay)
    self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=self.optimizer,
                                                  mode='min', factor=0.1,
                                                  patience=10, verbose=1, 
                                                  eps=1e-4)
    
    torch.manual_seed(self.seed)
    if torch.cuda.is_available():
      torch.cuda.manual_seed_all(self.seed)
    else: 
      torch.cuda.manual_seed(self.seed)
    torch.cuda.seed_all()
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    self.weight = torch.tensor(config.class_weight).to(self.device)
    self.criterion = torch.nn.CrossEntropyLoss(weight=self.weight,reduction='sum')
    # initialize the early_stopping object
    self.early_stopping = EarlyStopping(patience=config.patience, verbose=True)
    self.results = {}
    self.sgd_spv_matrix = {}
    self.eps = config.eps
    self.avg = config.avg

  def calc_accuracy(self, log_ps, labels):
    self.model.eval()
    ps = torch.exp(log_ps)
    top_p, top_class = ps.topk(1, dim=1)
    equals = top_class == labels.view(*top_class.shape)
    acc = torch.mean(equals.type(torch.FloatTensor))
    self.model.train()
    return acc

  def save_best_model(self, loss_val, acc_val, p, r, f, epoch, measure):
    weights_path_model = Path(f"{self.model_weights_path}/best_{measure}_{self.model_name}_{self.exp_num}_seed_{self.seed}.pth")
    print('saving_model best model: ')
    torch.save({'model_state_dict': self.model.state_dict(),
                'optimizer_state_dict': self.optimizer.state_dict(),
                'loss': loss_val,
                'accuracy': acc_val,
                'p':p,
                'r':r,
                'f':f}, weights_path_model)

  def save_checkpoint(self, loss_val, acc_val, p, r, f, epoch):
    weights_path_model = Path(f"{self.model_weights_path}/{self.model_name}_{epoch}_seed_{self.seed}.pth")
    if (epoch in self.save_points) and (not weights_path_model.exists()) and self.save_model:
        print('saving_model: ')
        torch.save({'model_state_dict': self.model.state_dict(),
                    'optimizer_state_dict': self.optimizer.state_dict(),
                    'loss': loss_val,
                    'accuracy': acc_val,
                    'p':p,
                    'r':r,
                    'f':f}, weights_path_model)

  def load_checkpoint(self, weights_path, epoch):
    checkpoint = torch.load(weights_path)
    self.model.load_state_dict(checkpoint['model_state_dict'])
    self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    self.model.to(self.device)
    loss = checkpoint['loss']
    accuracy = checkpoint['accuracy']
    # p = checkpoint['p']
    # r = checkpoint['r']
    # f = checkpoint['f']
    print(f"Uploaded weights succesfuly at epoch number {epoch}")
    return loss, accuracy

  def record(self, epoch, **kwargs):
    epoch = "{:02d}".format(epoch)
    temp = f"| epoch   # {epoch} :"
    for key, value in kwargs.items():
      key = f"{self.model_name}_{key}"
      if not self.results.get(key):
        self.results[key] = []
      self.results[key].append(value)
      val = '{:.4f}'.format(np.round(value, 4))
      temp += f"{key} : {val}      |       "
    print(temp)

  def clip_gradient(self, clip_value):
    params = list(filter(lambda p: p.grad is not None, self.model.parameters()))
    for p in params:
        p.grad.data.clamp_(-clip_value, clip_value)

  def fit(self, trainloader, validloader, config):
    is_loaded = False
    epoch = 1
    while epoch<self.epochs+1:
      results = []
      l_epoch = -1 if epoch==1 else epoch-1
      for i in self.save_points[::-1]:
        weights_path = Path(f"{self.model_weights_path}/{config.model_name}_{i}_seed_{self.seed}.pth")
        if (weights_path.exists()) and (self.upload_model) and (epoch<i) and (not is_loaded):
          epoch_train_loss, epoch_train_acc = self.load_checkpoint(weights_path, i)
          epoch = i+1
          l_epoch = i
          is_loaded = True
          break
      epoch_train_loss, epoch_train_acc, (p_train, r_train, f_train) = self.train_model(trainloader, epoch)
      epoch_valid_loss, epoch_valid_acc, (p_valid, r_valid, f_valid) = self.eval_model(validloader, epoch)
      
      #saving best model weights
      if epoch_valid_loss<self.best_loss:
        self.best_loss = epoch_valid_loss
        self.save_best_model(epoch_valid_loss, epoch_valid_acc, p_valid, r_valid, 
                      f_valid, epoch, 'val_loss')
      if epoch_valid_acc>self.best_acc:
        self.best_acc = epoch_valid_acc
        self.save_best_model(epoch_valid_loss, epoch_valid_acc, p_valid, r_valid, 
                      f_valid, epoch, 'val_acc')
      
      results.append([self.exp_num, config.model_name, datetime.datetime.now(), config.exp, epoch, epoch_train_loss, 
                     epoch_valid_loss,epoch_train_acc, epoch_valid_acc, p_train, p_valid, r_train, r_valid, f_train, f_valid])
      df = pd.DataFrame(results)
      df.to_csv(f'results/doc_embedding_{config.dataset}_results.csv', mode='a', header=False, index=False)
      self.scheduler.step(epoch_valid_loss)
      self.record(epoch, train_loss=epoch_train_loss,
                  validation_loss=epoch_valid_loss,
                  train_accuracy=epoch_train_acc,
                  validation_accuracy=epoch_valid_acc,
                  train_p=p_train,
                  validation_p=p_valid,
                  train_r=r_train,
                  validation_r=r_valid,
                  train_f=f_train,
                  validation_f=f_valid)
      self.save_checkpoint(epoch_train_loss, epoch_train_acc, p_train, r_train, 
                           f_train, epoch)
      self.early_stopping(epoch_valid_loss)
      if self.early_stopping.early_stop:
        print("Early stopping")
        break
      epoch += 1

  def caclculate_p_r_f(self, prediction):
    self.model.eval()
    predic_prob = self.soft(prediction)
    predicted_label = torch.argmax(predic_prob,axis=1)
    return predicted_label.cpu().detach().numpy()

  def train_model(self, train_iter, epoch):
    total_epoch_loss, total_epoch_acc, steps = 0, 0, 0
    self.model.train()
    train_loss, counter = 0, 0
    for idx, batch in enumerate(train_iter):
      features_data = []
      text, text_lengths = batch.text
      target = batch.label

      # add meta features
      for i, feat in enumerate(self.model.meta_feat_names):
        features_data.append(getattr(batch, feat))
      if len(features_data)!=0:
        features_data = torch.stack(features_data)
        features_data = features_data.to(self.device)

      target = torch.autograd.Variable(target).long()
      text = text.to(self.device)
      target = target.to(self.device)
      self.optimizer.zero_grad()
      prediction = self.model(text=text, text_lengths=text_lengths, meta_data_feat=features_data)
      loss = self.criterion(prediction, target)

      # calculate accuracy
      num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).float().sum()
      acc = 100.0 * num_corrects / len(batch)

      predicted_label = self.caclculate_p_r_f(prediction)
      (p,r,f,_) = precision_recall_fscore_support(target.cpu().detach().numpy() , predicted_label , average=self.avg)

      self.model.train()
      loss.backward()
      self.clip_gradient(1)
      self.optimizer.step()
      total_epoch_loss += loss.item() / len(batch)
      total_epoch_acc += acc.item()
      
    return total_epoch_loss / len(train_iter), total_epoch_acc / len(train_iter), (p,r,f)

  def eval_model(self, val_iter, epoch):
    total_epoch_loss, total_epoch_acc = 0, 0
    self.model.eval()
    with torch.no_grad():
      for idx, batch in enumerate(val_iter):
        features_data = []
        text, text_lengths = batch.text
        target = batch.label

        # add meta features
        for i, feat in enumerate(self.model.meta_feat_names):
          features_data.append(getattr(batch, feat))
        if len(features_data)!=0:
          features_data = torch.stack(features_data)
          features_data = features_data.to(self.device)

        target = torch.autograd.Variable(target).long()
        text = text.to(self.device)
        target = target.to(self.device)
        prediction = self.model(text=text, text_lengths=text_lengths, meta_data_feat=features_data)
        loss = self.criterion(prediction, target)

        # calculate accuracy
        num_corrects = (torch.max(prediction, 1)[1].view(target.size()).data == target.data).sum()
        acc = 100.0 * num_corrects / len(batch)
        total_epoch_loss += loss.item()  / len(batch)
        total_epoch_acc += acc.item()

        predicted_label = self.caclculate_p_r_f(prediction)
        (p,r,f,_) = precision_recall_fscore_support(target.cpu().detach().numpy() , predicted_label , average=self.avg)


    return total_epoch_loss / len(val_iter), total_epoch_acc / len(val_iter), (p,r,f)

# Run

## Configuration

In [0]:
class Config:
  """Holds model hyperparams and data information.
  The config class is used to store various hyperparameters and dataset
  information parameters. Model objects are passed a Config() object at
  instantiation.
  """
  def __init__(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

  def add_attributes(self, **kwargs):
    for key, value in kwargs.items():
      setattr(self, key, value)

def create_directories(l):
  for directory_path in l:
    if not (isdir(directory_path)):
      mkdir(directory_path)


def get_base_config():
  ROOT_PATH = 'models/doc_embedding/'
  model_weights_dir = f"{ROOT_PATH}model_weights"
  graphs_dir = f"{ROOT_PATH}graphs"
  SAVE_FIGS = True
  ####################################################################
  BATCH_SIZE = 128 
  hidden_size = 30
  embedding_length = 100
  epochs = 50
  ####################################################################
  # model consistency options
  SAVE_TO_CHECKPOINTS = True # if true, saves model.name_epcoch file into the weights folder
  LOAD_CHECKPOINTS = False # # if true, every epoch tries to load pretrained weights
  ####################################################################
  # if needed, can be modified to upload the 'best model'
  return Config(seed=2,
                # learning
                epochs=epochs, batch_size=BATCH_SIZE,
                eps=0.00001, patience=5,
                weight_decay=5e-4,momentum=0.9,lr=0.001,
                #model
                hidden_size=hidden_size, hidden_layer=1,
                seq_max_len=500, embedding_dim=embedding_length, embedding_type ='avg',
                #save
                save_points=np.arange(1,epochs/10+1)*10,
                save_model=SAVE_TO_CHECKPOINTS,
                upload_model=LOAD_CHECKPOINTS,
                #path
                model_weights_path=model_weights_dir,
                graphs_path = graphs_dir,
                device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

config = get_base_config()

### configure dataset

In [0]:
# define configuration for the different datasets
datasets = {"combined":{"root_path_data": r"data/preprocessed/ny_guardian_kaggle/combined_data_tokenized_shuffled.csv",
                      "origin_cols": ['label', 'date', 'source', 'text_original', 'title', "text"],
                      "n_classes":2, "has_valid":False, "class_weight":[0.3, 0.7], "avg":"binary","dropout":0.1},
              "liar":{"root_path_data": r"data/preprocessed/liar_dataset/",
                      "origin_cols": ['ID', 'label', 'text_original', 'class', 'text'],
                      "n_classes":6, "has_valid":True, "class_weight":[2/15, 2/15, 2/15, 2/15, 2/6, 2/15],
                      "avg":"micro","dropout":0.5},
              "liar_binary":{"root_path_data": r"data/preprocessed/liar_dataset/",
                      "origin_cols": ['ID', 'label_all', 'text_original', 'label', 'text'],
                      "n_classes":2, "has_valid":True, "class_weight":[0.3, 0.7],
                      "avg": "binary","dropout":0.3},
            "Buzz+PoliticalFact":{"root_path_data": r"data/preprocessed/Buzz+PoliticalFact/combined_data_tokenized.csv",
                      "origin_cols": ['text_original', 'title', 'url', 'publish_date', 'source', 'label','text'],
                      "n_classes":2, "has_valid":False, "class_weight":[0.5, 0.5], "avg":"binary",
                      "dropout":0.5}}          

### configure model

In [0]:
# define model
models = {"BiLSTM": BiLstmlClassification}

### configure doc embedding

In [0]:
doc_embedding_dict = {"self_attention": models["BiLSTM"].self_attention_net,
                      "attention": models["BiLSTM"].attention_net,
                      "pooling": models["BiLSTM"].pooling,
                      "n_gram_pooling": models["BiLSTM"].n_gram_pooling}

In [0]:
def get_model_embedding_type(model_name="pooling", embed_type='max'):
  if model_name=='pooling' or model_name=="n_gram_pooling":
    return embed_type
  else: 
    return ""

## Load data and Embedding

In [0]:
def load_dataset(path, glove_dim, doc_length, SEED, device, origin_cols, has_valid=False, debug=False):
    """
    tokenizer : Breaks sentences into a list of words. If sequential=False, no tokenization is applied
    Field : A class that stores information about the way of preprocessing
    fix_length : An important property of TorchText is that we can let the input to be variable length, and TorchText will
                 dynamically pad each sequence to the longest sequence in that "batch". But here we are using fix_length which
                 will pad each sequence to have a fix length of doc_length.

    build_vocab : It will first make a vocabulary or dictionary mapping all the unique words present in the train_data to an
                  idx and then after it will use GloVe word embedding to map the index to the corresponding word embedding.

    vocab.vectors : This returns a torch tensor of shape (vocab_size x embedding_dim) containing the pre-trained word embeddings.
    BucketIterator : Defines an iterator that batches examples of similar lengths together to minimize the amount of padding needed.

    """

    # loading custom dataset
    # initialize glove embeddings
    
    TEXT = data.Field(sequential=True, tokenize=lambda x: x.split(" "),
                      lower=False, include_lengths=True,
                      batch_first=True, fix_length=None, init_token="<SOA>", eos_token="<EOA>")
    NUMERICAL_FLOAT = data.Field(use_vocab=False,sequential=False, dtype=torch.float64)
    NUMERICAL_INT = data.Field(use_vocab=False,sequential=False, dtype=torch.int64)
    LABEL = data.LabelField(use_vocab=False,sequential=False)

    # label, date, source, text, title                    
    preprocessed_fields = [("article_length", NUMERICAL_FLOAT),
              ("total_sentences", NUMERICAL_FLOAT),("avg_sent_length", NUMERICAL_FLOAT),
              ("word_count", NUMERICAL_FLOAT), ("number_count", NUMERICAL_FLOAT),
              ("avg_word_length", NUMERICAL_FLOAT), ("count_pos_adj", NUMERICAL_FLOAT), 
              ("count_pos_sl", NUMERICAL_FLOAT), ("count_exclamation", NUMERICAL_FLOAT), 
              ("count_date", NUMERICAL_FLOAT), ("allCapsCount", NUMERICAL_FLOAT),
              ("catLaw", NUMERICAL_FLOAT),("catCommunication", NUMERICAL_FLOAT),
              ("catCrime", NUMERICAL_FLOAT),("catPayment", NUMERICAL_FLOAT),
              ("catPhone", NUMERICAL_FLOAT), ("catBanking", NUMERICAL_FLOAT),
              ("catWar", NUMERICAL_FLOAT),("catEconomics", NUMERICAL_FLOAT),
              ("catPolitics", NUMERICAL_FLOAT), ("catLeader", NUMERICAL_FLOAT), 
              ("catSocialMedia", NUMERICAL_FLOAT),("catSchool", NUMERICAL_FLOAT),
              ("catGovernment", NUMERICAL_FLOAT),("catMoney", NUMERICAL_FLOAT), 
              ("catWork", NUMERICAL_FLOAT),("catSpeaking", NUMERICAL_FLOAT),
              ("catInternet", NUMERICAL_FLOAT),("catBusiness", NUMERICAL_FLOAT),
              ("sentNeg", NUMERICAL_FLOAT),("sentNeutral", NUMERICAL_FLOAT),
              ("sentPos", NUMERICAL_FLOAT), ("sentCompound", NUMERICAL_FLOAT)]

    dict_fields = {'label':LABEL,
                   'text':TEXT}
    original_fields = [(col, dict_fields.get(col)) for col in origin_cols]
    fields = original_fields + preprocessed_fields
    
    if not has_valid:
      train_data = data.TabularDataset(path=path, format='csv', fields=fields, skip_header=True)
      train_data, valid_data = train_data.split(split_ratio=0.8, random_state=random.seed(SEED))
    else:
      train_data, valid_data = data.TabularDataset.splits(path=path, train="train_tokenized.csv", 
                                                          validation="valid_tokenized.csv", format="csv",
                                                          skip_header=True, fields=fields)

    # Further splitting of training_data to create new training_data & validation_data
    train_iter, valid_iter = data.BucketIterator.splits((train_data, valid_data), batch_size=128,
                                                        sort_key=lambda x: len(x.text), repeat=False,
                                                        shuffle=True)
    # Initialized embedding
    if not os.path.exists(".vector_cache"):
        os.mkdir(".vector_cache")

    TEXT.build_vocab(train_data, min_freq=3,
                     vectors=GloVe(name='6B',
                                   dim=glove_dim))
    LABEL.build_vocab(train_data)

    word_embeddings = TEXT.vocab.vectors  # pretrained_embedding
    vocab_size = len(TEXT.vocab)

    if debug:
      # printing
      print("Length of Text Vocabulary: " + str(len(TEXT.vocab)))  # No. of unique tokens in text
      print("Vector size of Text Vocabulary: ", TEXT.vocab.vectors.size())
      print("Label Length: " + str(len(LABEL.vocab)))  # No. of unique tokens in label
      print(TEXT.vocab.freqs.most_common(10))  # Commonly used words
      print(TEXT.vocab.stoi)  # Word dictionary

    return vocab_size, word_embeddings, train_iter, valid_iter, TEXT.vocab.stoi['<pad>']

## Experiments

In [0]:
create_directories([config.graphs_path,config.model_weights_path])

In [14]:
# run model
model_name = "BiLSTM"
dataset_list = [#'liar',
                #'liar_binary',
                'Buzz+PoliticalFact'
                ]
doc_embedding_list = ["self_attention",
                      "attention",
                      "pooling"
                      ,"n_gram_pooling"
                      ]
lr = [0.001]
hidden_size = [100]
hp_comb = list(itertools.product(lr, hidden_size))
for selected_dataset in dataset_list:
  config.add_attributes(**datasets[selected_dataset], dataset=selected_dataset)
  #load dataset and embedding
  vocab_size, word_embeddings, train_iter, valid_iter, pad_index =\
      load_dataset(config.root_path_data, config.embedding_dim, 
                  config.seq_max_len, config.seed, config.device,
                  config.origin_cols, config.has_valid)
  for embedding_name in doc_embedding_list:
    embd_type = get_model_embedding_type(embedding_name,"avg")
    for exp_num in np.arange(3):
      for l, h in hp_comb:
        print('lr', l, 'hidden', h, 'drop_ratio', config.dropout)
        exp_name = f'{model_name}_{embedding_name}_{embd_type}_{selected_dataset}'
        config.add_attributes(model_name=exp_name, 
                              lr = l,
                              weight_decay = 0.00005,
                              hidden_size=h,
                              exp=f'{l},{h},{config.dropout}',
                              doc_embedding = doc_embedding_dict[embedding_name],
                              embedding_type=embd_type,
                              exp_num=exp_num)
        model = models[model_name](config=config, pretrained_embeddings=word_embeddings, pad_index=pad_index)
        print(f"{'#'*50} \n running {config.model_name} experiment\n{'#'*50} \n") 
        trainer = Trainer(model, config)
        trainer.fit(train_iter, valid_iter, config)

lr 0.001 hidden 100 drop_ratio 0.5
################################################## 
 running BiLSTM_self_attention__Buzz+PoliticalFact experiment
################################################## 



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


saving_model best model: 
saving_model best model: 
| epoch   # 01 :BiLSTM_self_attention__Buzz+PoliticalFact_train_loss : 0.3706      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_loss : 0.3353      |       BiLSTM_self_attention__Buzz+PoliticalFact_train_accuracy : 44.8107      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_accuracy : 52.3810      |       BiLSTM_self_attention__Buzz+PoliticalFact_train_p : 0.0000      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_p : 0.0000      |       BiLSTM_self_attention__Buzz+PoliticalFact_train_r : 0.0000      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_r : 0.0000      |       BiLSTM_self_attention__Buzz+PoliticalFact_train_f : 0.0000      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_f : 0.0000      |       
| epoch   # 02 :BiLSTM_self_attention__Buzz+PoliticalFact_train_loss : 0.3424      |       BiLSTM_self_attention__Buzz+PoliticalFact_validation_loss : 0.3659      |